In [1]:
#installing num2words library
!pip install num2words

In [2]:
#importing packages
import gzip
import shutil
import json
from pprint import pprint
import gensim
import nltk
import re
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Make sure you have NLTK's stopwords downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
nltk.download('punkt')
from num2words import num2words
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#imports the NLTK library, downloads the NLTK stopwords dataset, and then imports the stopwords dataset from the NLTK corpus.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import pandas as pd
import numpy as np


import string
import re
from pprint import pprint

# NLTK

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) #this depends on each language
stop_words.add("rs")
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization


# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [5]:
#download and install the spaCy English language model "en_core_web_sm".
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
#read data from file into a dataframe using pandas.
df = pd.read_csv("/content/cs_domain_final.csv")

In [7]:
#display column names of dataframe.
df.columns

Index(['paperId', 'id', 'title', 'fieldsOfStudy', 'source', 'version', 'added',
       'created', 'text'],
      dtype='object')

In [8]:
#replace newline characters with spaces in the 'text' column of the DataFrame 'df'.
df["text"] = df["text"].apply(lambda x: str(x).replace('\n', ' '))

In [9]:
#download the NLTK 'punkt' and 'stopwords' resources and import necessary modules for text processing.
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Text Preprocessing

In [10]:
#define a function 'tokenize_sentences' that tokenizes input text into sentences using NLTK's 'sent_tokenize' method.
def tokenize_sentences(text):
    return sent_tokenize(text)

In [11]:
#tokenize the 'text' column into sentences and store the result in a new 'sentences' column in the DataFrame 'df'
df['sentences'] = df['text'].apply(tokenize_sentences)

In [12]:
#define a function 'remove_stopwords_from_list' that removes stopwords from a list of sentences and returns the cleaned sentences.
def remove_stopwords_from_list(sentences):
    stop_words = set(stopwords.words('english'))
    cleaned_sentences = []

    for sentence in sentences:
        words = sentence.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        cleaned_sentences.append(' '.join(filtered_words))

    return cleaned_sentences

In [13]:
#apply the 'remove_stopwords_from_list' function to remove stopwords from the 'sentences' column in the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_stopwords_from_list)

In [14]:
#define a function 'lemmatize' that lemmatizes words in a list of sentences using NLTK's WordNetLemmatizer.
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def lemmatize(sentences):
    lemmatizer = WordNetLemmatizer()
    sent_list = []

    for sent in sentences:
        words = word_tokenize(sent)
        words = [lemmatizer.lemmatize(word) for word in words]
        words = " ".join(words)
        sent_list.append(words)

    return sent_list


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
#apply the 'lemmatize' function to lemmatize words in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(lemmatize)

In [16]:
#remove URLs from a list of sentences using regular expressions and store the cleaned sentences.
def remove_urls_from_list(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Use a regular expression to find and remove URLs
            sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [17]:
#apply the 'remove_urls_from_list' function to remove urls from the beginning of sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_urls_from_list)

In [18]:
#remove bullet points ('*' or '•') at the beginning of each sentence in a list of sentences.
def remove_bullet_points_from_list(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Remove bullet points ('*' or '•') at the beginning of each sentence
            sentence = re.sub(r'^\s*[\*\•]+\s*', '', sentence)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [19]:
#apply the 'remove_bullet_points_from_list' function to remove bullet points from the beginning of sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_bullet_points_from_list)

In [20]:
#remove apostrophes (' and ’) from sentences in a list of sentences.
def remove_apostrophes_from_list(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Use a regular expression to remove apostrophes
            sentence = re.sub(r"[\']", '', sentence)
            sentence = re.sub(r"['’]", '', sentence)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [21]:
#apply the 'remove_apostrophes_from_list' function to remove apostrophes from sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_apostrophes_from_list)

In [22]:
#remove hyphens between words in sentences in a list of sentences.
def remove_hyphens_from_list(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Remove hyphens from the sentence
            sentence = re.sub(r'(\w+)-(\w+)', r'\1 \2', sentence)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [23]:
#apply the 'remove_hypens_from_list' function to remove hypens from the beginning of sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_hyphens_from_list)

In [24]:
#remove enumerations (e.g., numbers followed by periods) from sentences in a list of sentences.
def remove_enumerations_from_list(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Remove enumerations from the sentence
            sentence = re.sub(r'^\s*\d+\.\s*', '', sentence)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [25]:
#apply the 'remove_enumerations_from_list' function to remove enumerations from sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_enumerations_from_list)

In [26]:
#convert numerical values to their text representations in sentences in a list of sentences.
def numerical_to_text_conversion(sentences_list):
    cleaned_sentences = []

    for sentence in sentences_list:
        if isinstance(sentence, str):
            # Find and replace numerical values with their text representations
            sentence = re.sub(r'\b\d+\b', lambda match: num2words(int(match.group())), sentence)
        cleaned_sentences.append(sentence)

    return cleaned_sentences

In [27]:
#apply the 'numerical_to_text_conversion' function to convert numerical values to their text representations in sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(numerical_to_text_conversion)

In [28]:
#remove punctuation from sentences in a list of sentences.
def remove_punctuation(sentences):
  sent_list=[]
  for sent in sentences:
    words=word_tokenize(sent)
    str=""
    for i in range(len(words)):
      x=re.sub("[^\w\s]|_","",words[i]).strip()
      str=str+" "+x
      str=str.strip()
    sent_list.append(str)
  return sent_list


In [29]:
#apply the 'remove_punctuation' function to remove punctuation from sentences in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(remove_punctuation)

In [30]:
#convert sentences to lowercase in a list of sentences.
def lowercase_sentences(sentences):
    # Convert each sentence to lowercase
    lowercased_sentences = [sentence.lower() for sentence in sentences]
    return lowercased_sentences

In [31]:
#apply the 'lowercase_sentences' function to convert sentences to lowercase in the 'sentences' column of the DataFrame 'df'.
df['sentences'] = df['sentences'].apply(lowercase_sentences)

Spilitting of train and test data

In [32]:
#split the DataFrame 'df' into a training set 'train' containing the first 35 rows and a test set 'test' containing rows 3000 to 3999.
train = df[:30]
test = df[3000:4000]

In [33]:
#import the 'math' module and the 'defaultdict' class from the 'collections' module.
import math
from collections import defaultdict

In [34]:
#convert the 'sentences' column of the 'train' DataFrame to a list and print it.
sentences = train['sentences'].tolist()
print(sentences)

[['complete targets coverage energy harvesting internet things ambient backscatter article considers deriving set cover set active node responsible monitoring target internet things iot network', 'key distinction prior work article considers sensor node aided backscatter communication allow communicate negligible energy cost using ambient radio frequency rf signal', 'article contains three main novelty', 'first present mixed integer linear program milp used compute global optimal solution', 'second also outline centralized greedy scheduling cgs algorithm selects node based energy number covered target', 'third present distributed greedy scheduling dgs algorithm selects node according energy level', 'simulation result show equipping node ambient backscattering capability increase ratio complete target coverage one hundred compared existing technique'], ['nonorthogonal multiple access enabled two way relay system using signal alignment conventional two way relay nonorthogonal multiple ac

In [35]:
#calculate and print the total number of sentences in the 'sentences' list.
sentence_counts = 0

for inner_list in sentences:
    sentence_count = len(inner_list)
    sentence_counts += sentence_count

print(sentence_counts)

215


In [36]:
#extract information about each sentence, including its position in nested lists and word count, and store the results in 'sent_info'.
from nltk.tokenize import word_tokenize

def sentence_information(sentences):
    sent_info = []
    for outer_idx, inner_sentences in enumerate(sentences):
        for inner_idx, sent in enumerate(inner_sentences):
            words = word_tokenize(sent)

            sent_dict = {"outer_idx": outer_idx, "inner_idx": inner_idx, "sent_word_count": len(words)}
            sent_info.append(sent_dict)

    return sent_info

sent_info=sentence_information(sentences)
print(sent_info)

[{'outer_idx': 0, 'inner_idx': 0, 'sent_word_count': 24}, {'outer_idx': 0, 'inner_idx': 1, 'sent_word_count': 22}, {'outer_idx': 0, 'inner_idx': 2, 'sent_word_count': 5}, {'outer_idx': 0, 'inner_idx': 3, 'sent_word_count': 12}, {'outer_idx': 0, 'inner_idx': 4, 'sent_word_count': 15}, {'outer_idx': 0, 'inner_idx': 5, 'sent_word_count': 12}, {'outer_idx': 0, 'inner_idx': 6, 'sent_word_count': 18}, {'outer_idx': 1, 'inner_idx': 0, 'sent_word_count': 31}, {'outer_idx': 1, 'inner_idx': 1, 'sent_word_count': 16}, {'outer_idx': 1, 'inner_idx': 2, 'sent_word_count': 6}, {'outer_idx': 1, 'inner_idx': 3, 'sent_word_count': 12}, {'outer_idx': 1, 'inner_idx': 4, 'sent_word_count': 15}, {'outer_idx': 1, 'inner_idx': 5, 'sent_word_count': 7}, {'outer_idx': 1, 'inner_idx': 6, 'sent_word_count': 13}, {'outer_idx': 1, 'inner_idx': 7, 'sent_word_count': 7}, {'outer_idx': 1, 'inner_idx': 8, 'sent_word_count': 13}, {'outer_idx': 1, 'inner_idx': 9, 'sent_word_count': 17}, {'outer_idx': 1, 'inner_idx': 10, 

In [37]:
#calculate the frequency of words in each sentence and store the results in 'freq_info'. Then, print 'freq_info'.
def frequency_of_words(sentences):
    freq_info = []
    for outer_idx, inner_sentences in enumerate(sentences):
        for inner_idx, sent in enumerate(inner_sentences):
            words = word_tokenize(sent)
            frequency_dict = {}
            for word in words:
                if word in frequency_dict.keys():
                    frequency_dict[word] = frequency_dict[word] + 1
                else:
                    frequency_dict[word] = 1
            temp = {"outer_idx": outer_idx, "inner_idx": inner_idx, "frequency_of_word": frequency_dict}
            freq_info.append(temp)

    return freq_info

freq_info = frequency_of_words(sentences)
print(freq_info)

[{'outer_idx': 0, 'inner_idx': 0, 'frequency_of_word': {'complete': 1, 'targets': 1, 'coverage': 1, 'energy': 1, 'harvesting': 1, 'internet': 2, 'things': 2, 'ambient': 1, 'backscatter': 1, 'article': 1, 'considers': 1, 'deriving': 1, 'set': 2, 'cover': 1, 'active': 1, 'node': 1, 'responsible': 1, 'monitoring': 1, 'target': 1, 'iot': 1, 'network': 1}}, {'outer_idx': 0, 'inner_idx': 1, 'frequency_of_word': {'key': 1, 'distinction': 1, 'prior': 1, 'work': 1, 'article': 1, 'considers': 1, 'sensor': 1, 'node': 1, 'aided': 1, 'backscatter': 1, 'communication': 1, 'allow': 1, 'communicate': 1, 'negligible': 1, 'energy': 1, 'cost': 1, 'using': 1, 'ambient': 1, 'radio': 1, 'frequency': 1, 'rf': 1, 'signal': 1}}, {'outer_idx': 0, 'inner_idx': 2, 'frequency_of_word': {'article': 1, 'contains': 1, 'three': 1, 'main': 1, 'novelty': 1}}, {'outer_idx': 0, 'inner_idx': 3, 'frequency_of_word': {'first': 1, 'present': 1, 'mixed': 1, 'integer': 1, 'linear': 1, 'program': 1, 'milp': 1, 'used': 1, 'comput

In [38]:
#find unique words in the 'sentences' list and print the count of unique words.
def unik_word(sentences):
    unik = []
    for outer_sentences in sentences:
        for sent in outer_sentences:
            words = word_tokenize(sent)
            for word in words:
                if word not in unik:
                    unik.append(word)
    return unik

unik_words = unik_word(sentences)
print(len(unik_words))

1663


In [39]:
#import the 'lil_matrix' and 'csr_matrix' classes from 'scipy.sparse' and the 'numpy' library as 'np'.
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

Calculation of TF Values

In [40]:
#calculate TF (Term Frequency) values for each word in each sentence using the provided information and store the results in 'tf_values'. Then, print 'tf_values'.
def tf_calculation(sent_info, freq_info, unik_words):
    tf_values = lil_matrix((len(sent_info), len(unik_words)), dtype=np.float32)

    for i, freq_info_dict in enumerate(freq_info):
        for word, freq in freq_info_dict["frequency_of_word"].items():
            tf = freq / sent_info[i]["sent_word_count"]
            tf_values[i, unik_words.index(word)] = tf

    return tf_values.tocsr()

tf_values=tf_calculation(sent_info,freq_info,unik_words)
print(tf_values)

  (0, 0)	0.041666668
  (0, 1)	0.041666668
  (0, 2)	0.041666668
  (0, 3)	0.041666668
  (0, 4)	0.041666668
  (0, 5)	0.083333336
  (0, 6)	0.083333336
  (0, 7)	0.041666668
  (0, 8)	0.041666668
  (0, 9)	0.041666668
  (0, 10)	0.041666668
  (0, 11)	0.041666668
  (0, 12)	0.083333336
  (0, 13)	0.041666668
  (0, 14)	0.041666668
  (0, 15)	0.041666668
  (0, 16)	0.041666668
  (0, 17)	0.041666668
  (0, 18)	0.041666668
  (0, 19)	0.041666668
  (0, 20)	0.041666668
  (1, 3)	0.045454547
  (1, 7)	0.045454547
  (1, 8)	0.045454547
  (1, 9)	0.045454547
  :	:
  (211, 1654)	0.045454547
  (211, 1655)	0.045454547
  (211, 1656)	0.045454547
  (211, 1657)	0.045454547
  (211, 1658)	0.045454547
  (211, 1659)	0.045454547
  (211, 1660)	0.045454547
  (212, 200)	0.2
  (212, 330)	0.2
  (212, 366)	0.2
  (212, 633)	0.2
  (212, 1661)	0.2
  (213, 112)	0.14285715
  (213, 200)	0.14285715
  (213, 329)	0.14285715
  (213, 355)	0.14285715
  (213, 390)	0.14285715
  (213, 1059)	0.14285715
  (213, 1662)	0.14285715
  (214, 151)	0.16666

Calculation of IDF Values

In [41]:
#calculate IDF (Inverse Document Frequency) values for each word in each sentence using the provided information and store the results in 'idf_values'. Then, print 'idf_values'.
def idf_calculate(sent_info, freq_info, unik_words):
    idf_values = lil_matrix((len(sent_info), len(unik_words)), dtype=np.float32)

    num_sentences = len(sent_info)
    word_sum_dict = {word: sum(freq_info_dict["frequency_of_word"].get(word, 0) for freq_info_dict in freq_info) for word in unik_words}

    for i, freq_info_dict in enumerate(freq_info):
        for word in freq_info_dict["frequency_of_word"]:
            idf = math.log(num_sentences / (1 + word_sum_dict[word]))
            idf_values[i, unik_words.index(word)] = idf

    return idf_values.tocsr()

idf_values=idf_calculate(sent_info,freq_info,unik_words)
print(idf_values)

  (0, 0)	3.7612002
  (0, 1)	4.6774907
  (0, 2)	4.2720256
  (0, 3)	2.8056886
  (0, 4)	4.6774907
  (0, 5)	3.7612002
  (0, 6)	3.9843438
  (0, 7)	3.9843438
  (0, 8)	4.2720256
  (0, 9)	2.7315807
  (0, 10)	4.2720256
  (0, 11)	4.6774907
  (0, 12)	2.7315807
  (0, 13)	4.6774907
  (0, 14)	3.7612002
  (0, 15)	3.2911966
  (0, 16)	4.6774907
  (0, 17)	3.424728
  (0, 18)	3.5788786
  (0, 19)	3.424728
  (0, 20)	2.4802663
  (1, 3)	2.8056886
  (1, 7)	3.9843438
  (1, 8)	4.2720256
  (1, 9)	2.7315807
  :	:
  (211, 1654)	4.6774907
  (211, 1655)	4.6774907
  (211, 1656)	4.6774907
  (211, 1657)	4.6774907
  (211, 1658)	4.6774907
  (211, 1659)	4.6774907
  (211, 1660)	4.6774907
  (212, 200)	3.068053
  (212, 330)	2.3749058
  (212, 366)	3.1734135
  (212, 633)	3.424728
  (212, 1661)	4.6774907
  (213, 112)	2.426199
  (213, 200)	3.068053
  (213, 329)	2.3749058
  (213, 355)	3.9843438
  (213, 390)	3.9843438
  (213, 1059)	2.8056886
  (213, 1662)	4.6774907
  (214, 151)	3.068053
  (214, 249)	4.2720256
  (214, 354)	3.1734135

Calculation of TF-IDF Values

In [42]:
#calculate TF-IDF (Term Frequency-Inverse Document Frequency) values by multiplying TF and IDF matrices and store the results in 'tfidf_values'. Then, print 'tfidf_values'.
def tfidf_calculation(tf_values, idf_values):
    tfidf_values = tf_values.multiply(idf_values)
    return tfidf_values

tfidf_values=tfidf_calculation(tf_values,idf_values)
print(tfidf_values)

  (0, 0)	0.15671667
  (0, 1)	0.19489545
  (0, 2)	0.17800108
  (0, 3)	0.11690369
  (0, 4)	0.19489545
  (0, 5)	0.31343335
  (0, 6)	0.33202866
  (0, 7)	0.16601433
  (0, 8)	0.17800108
  (0, 9)	0.11381587
  (0, 10)	0.17800108
  (0, 11)	0.19489545
  (0, 12)	0.22763173
  (0, 13)	0.19489545
  (0, 14)	0.15671667
  (0, 15)	0.1371332
  (0, 16)	0.19489545
  (0, 17)	0.142697
  (0, 18)	0.14911994
  (0, 19)	0.142697
  (0, 20)	0.10334443
  (1, 3)	0.1275313
  (1, 7)	0.18110654
  (1, 8)	0.19418299
  (1, 9)	0.12416276
  :	:
  (211, 1654)	0.21261322
  (211, 1655)	0.21261322
  (211, 1656)	0.21261322
  (211, 1657)	0.21261322
  (211, 1658)	0.21261322
  (211, 1659)	0.21261322
  (211, 1660)	0.21261322
  (212, 200)	0.6136106
  (212, 330)	0.47498116
  (212, 366)	0.6346827
  (212, 633)	0.6849456
  (212, 1661)	0.9354982
  (213, 112)	0.34659988
  (213, 200)	0.4382933
  (213, 329)	0.3392723
  (213, 355)	0.569192
  (213, 390)	0.569192
  (213, 1059)	0.4008127
  (213, 1662)	0.668213
  (214, 151)	0.51134217
  (214, 249)

In [43]:
# import numpy as np

# def tf_calculation(sent_info,freq_info):
#   tf_values=np.zeros(shape=(len(sent_info),len(unik_words)))
#   for i,freq_info_dict in enumerate(freq_info):
#     for word in freq_info_dict["frequency_of_word"].keys():
#       tf=freq_info_dict["frequency_of_word"][word]/sent_info[i]["sent_word_count"]
#       tf_values[i][unik_words.index(word)]=tf
#   return tf_values


# tf_values=tf_calculation(sent_info,freq_info)
# print(tf_values)

In [44]:
# import math

# def idf_calculate(sent_info,freq_info):
#   idf_values=np.zeros(shape=(len(sent_info),len(unik_words)))
#   for i,freq_info_dict in enumerate(freq_info):
#     for word in freq_info_dict["frequency_of_word"].keys():
#       sum=0
#       for word_in_dict in freq_info:
#         if word in set(word_in_dict["frequency_of_word"].keys()):
#           sum=sum+word_in_dict["frequency_of_word"][word]
#       idf_values[i][unik_words.index(word)]=math.log(len(sentences)/sum)
#   return idf_values

# idf_values=idf_calculate(sent_info,freq_info)
# print(idf_values)

In [45]:
# def tfidf_calculation(tf_values,idf_values):
#   tfidf_values=np.zeros(shape=(len(sent_info),len(unik_words)))
#   tfidf_values=np.multiply(tf_values,idf_values)
#   return tfidf_values

# tfidf_values=tfidf_calculation(tf_values,idf_values)
# print(tfidf_values)


In [46]:
#print the shape (dimensions) of the TF-IDF values matrix.
print(tfidf_values.shape)

(215, 1663)


Initializing Context Matrix using tf-idf based vector modeling




In [47]:
#convert the TF-IDF values matrix 'tfidf_values' to a dense NumPy array and store it in 'context_matrix'.
context_matrix = tfidf_values.toarray()

In [48]:
#importing packages
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import defaultdict

In [49]:
#set the 'device' to use GPU ("cuda") if available; otherwise, use CPU ("cpu").
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
from collections import Counter, defaultdict
word_freq = defaultdict(int)
for sentence in sentences:
    for token in sentence:
        word_freq[token] += 1

min_word_frequency = 1
filtered_word_freq = {word: freq for word, freq in word_freq.items() if freq >= min_word_frequency}
# Sort words by frequency in descending order
sorted_word_freq = dict(sorted(filtered_word_freq.items(), key=lambda item: item[1], reverse=True))

# Create the vocabulary based on the sorted frequencies
vocabulary = list(sorted_word_freq.keys())

# Create word-to-index and index-to-word mappings
word2idx = {word: idx for idx, word in enumerate(vocabulary)}
idx2word = {idx: word for idx, word in enumerate(vocabulary)}

total_word_count = sum(filtered_word_freq.values())
word_probabilities = [freq / total_word_count for freq in sorted_word_freq.values()]

# Convert word probabilities to a tensor
word_probabilities_tensor = torch.Tensor(word_probabilities).to(device)

# Get the vocabulary size
vocabulary_size = len(vocabulary)

In [51]:
#display vocabulary size
vocabulary_size

215

Skipgram Word Embeddings

In [52]:
#define a SkipGramBatchModel class for skip-gram word embeddings using PyTorch.
class SkipGramBatchModel(nn.Module):
    def __init__(self, device, vocabulary_size, embedding_dim, neg_num=5):
        super(SkipGramBatchModel, self).__init__()
        self.device = device
        self.neg_num = neg_num
        self.target_embeddings = nn.Embedding(vocabulary_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocabulary_size, embedding_dim)
        self.target_embeddings.weight.data.uniform_(-0.5 / embedding_dim, 0.5 / embedding_dim)
        self.context_embeddings.weight.data.uniform_(-0.5 / embedding_dim, 0.5 / embedding_dim)

    def forward(self, centers, contexts):
        u_embeds = self.target_embeddings(centers)
        v_embeds = self.context_embeddings(contexts)
        scores = torch.bmm(u_embeds, v_embeds.transpose(1, 2)).squeeze()
        return scores
    def get_embeddings(self):
        return self.target_embeddings.weight.data, self.context_embeddings.weight.data


In [53]:
#generate batches of context and target matrices from the context_matrix for training a word embedding model.
def generate_batch(context_matrix, batch_size=8):
    num_sentences, sentence_length = context_matrix.shape
    indices = np.arange(num_sentences)
    np.random.shuffle(indices)

    for i in range(0, num_sentences, batch_size):
        batch_indices = indices[i:i+batch_size]
        contexts = context_matrix[batch_indices]

        # Create the target matrix by shifting the context matrix by one word
        targets = np.roll(contexts, shift=-1, axis=1)

        yield torch.LongTensor(contexts).to(device), torch.LongTensor(targets).to(device)

In [54]:
#train the given model using the provided data for a specified number of epochs and return the learned word embeddings.
def train_batch(model, data, num_epochs=5, learning_rate=0.025, batch_size=8):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        total_loss = 0
        for contexts, targets in generate_batch(data, batch_size=batch_size):
            optimizer.zero_grad()
            scores = model(contexts, targets)
            loss = loss_fn(scores, torch.ones_like(scores))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")
    return model.get_embeddings()




In [55]:
#train a SkipGramBatchModel to learn word embeddings using context_matrix data with specific hyperparameters.
embedding_dim = 100
vocabulary_size = len(vocabulary)
model = SkipGramBatchModel(device, vocabulary_size, embedding_dim)

word_embeddings, context_embeddings = train_batch(model, context_matrix, num_epochs=10, learning_rate=0.025, batch_size=128)

Epoch 1/10, Loss: 1.3813509345054626
Epoch 2/10, Loss: 1.227132797241211
Epoch 3/10, Loss: 0.9412875771522522
Epoch 4/10, Loss: 0.5624028891324997
Epoch 5/10, Loss: 0.24632269889116287
Epoch 6/10, Loss: 0.07869831286370754
Epoch 7/10, Loss: 0.020718513056635857
Epoch 8/10, Loss: 0.005279654986225069
Epoch 9/10, Loss: 0.0014431329909712076
Epoch 10/10, Loss: 0.0004430811677593738


In [56]:
import pickle
#serialize and save word embeddings and their corresponding dictionary to a binary file.
def save_embeddings(filename="embeddings.bin", embeddings=None, dictionary=None):
    """Embeddings and reverse dictionary serialization and dump to a file."""
    if embeddings is not None:
        # Reshape the embeddings to (sentences * words, embedding_dim)
        flattened_embeddings = embeddings.reshape(-1, embeddings.shape[-1])

        data = {
            'emb': flattened_embeddings,
            'dict': dictionary
        }

        with open(filename, 'wb') as file:
            print("Saving embeddings to file:", filename)
            pickle.dump(data, file)
    else:
        print("Embeddings are None. Nothing to save.")

# Example usage:
save_embeddings(filename="batchwise_embeddings.bin", embeddings=word_embeddings, dictionary=word2idx)

Saving embeddings to file: batchwise_embeddings.bin


Defining a Word2Vec Class


In [99]:
import pickle
import numpy as np
#Word2Vec class for loading and using word embeddings, computing sentence similarity, and generating next sentences.
class Word2Vec(object):
    def __init__(self):
        self.embeddings = None
        self.dictionary = None
        self.reverse_dictionary = None

    def from_file(self, filename):
        file = open(filename, 'rb')
        data = pickle.load(file)
        print(data)
        self.embeddings = data['emb']
        self.dictionary = data['dict']
        self.reverse_dictionary = {v: k for k, v in self.dictionary.items()}

    def from_object(self, embeddings, dictionary):
        self.embeddings = embeddings
        self.dictionary = dictionary
        self.reverse_dictionary = {v: k for k, v in dictionary.items()}

    def inference(self, sentence):
        assert self.embeddings is not None and self.dictionary is not None, \
            'Embeddings not initialized, use from_file or from_object to load data.'
        sentence_idx = self.dictionary.get(sentence)
        # Unknown sentence returns UNK's sentence_idx
        if sentence_idx is None:
            sentence_idx = 0
        return self.embeddings[sentence_idx]

    def similarity(self, sentence1, sentence2):
        v1 = self.inference(sentence1)
        v2 = self.inference(sentence2)
        # Perform cosine similarity using torch
        cosine_similarity = torch.nn.functional.cosine_similarity(v1, v2, dim=0)
        return cosine_similarity.item()

    def most_similar(self, sentence, topk=10):
      assert self.embeddings is not None and self.dictionary is not None, \
          'Embeddings not initialized, use from_file or from_object to load data.'

      # Get the embedding for the input sentence
      input_embedding = self.inference(sentence)

      # Calculate cosine similarities between the input sentence and all other sentences
      similarities = []
      input_norm = np.linalg.norm(input_embedding)  # Calculate the norm of the input embedding
      for idx in range(len(self.embeddings)):
          if idx != 0:  # Skip the unknown sentence
              other_embedding = self.embeddings[idx]
              other_norm = np.linalg.norm(other_embedding)  # Calculate the norm of the other embedding
              if input_norm != 0 and other_norm != 0:
                  similarity = np.dot(input_embedding, other_embedding) / (input_norm * other_norm)
                  similarities.append((self.reverse_dictionary.get(idx, ""), similarity))

      # Sort the sentences by similarity in descending order
      similarities.sort(key=lambda x: x[1], reverse=True)

      # Get the top-k most similar sentences and their similarities
      topk_results = similarities[:topk]

      return topk_results

    def generate_next_sentence(self, input_sentence):
        most_similar_sentences = self.most_similar(input_sentence, topk=10)
        if most_similar_sentences:
            similar_sentence = most_similar_sentences[0][0]
            return similar_sentence.split()  # Split the similar sentence into words
        else:
            return []


In [100]:
#initialize a Word2Vec instance 'wv' and load word embeddings from the specified file.
wv = Word2Vec()
wv.from_file("/content/batchwise_embeddings.bin")


{'emb': tensor([[ 3.9987e-01, -1.9919e-01, -2.0536e-01,  ...,  2.4661e-01,
          3.9974e-01, -6.4620e-02],
        [ 2.7921e-01, -1.8047e-01, -6.6184e-02,  ...,  9.4707e-02,
          2.8297e-01, -1.2112e-01],
        [ 4.6208e-03,  4.3982e-03,  2.2192e-03,  ...,  4.1203e-04,
         -1.7552e-03,  4.1399e-03],
        ...,
        [-1.4523e-03, -5.6460e-04,  9.7506e-04,  ..., -3.4152e-04,
         -1.2712e-03, -2.3757e-03],
        [-1.9478e-03, -2.9548e-03, -4.7510e-03,  ...,  2.2883e-03,
         -2.2760e-03, -4.6729e-03],
        [ 4.3906e-03, -3.4374e-04,  3.8812e-03,  ...,  4.6459e-03,
          4.1093e-03, -4.4534e-03]]), 'dict': {'complete targets coverage energy harvesting internet things ambient backscatter article considers deriving set cover set active node responsible monitoring target internet things iot network': 0, 'key distinction prior work article considers sensor node aided backscatter communication allow communicate negligible energy cost using ambient radio fr

In [112]:
#retrieve sentences from the 'sentences' column of the DataFrame 'test' at index 3900.
sentences1 = test['sentences'][3000]
sentences1

['detecting fake co visitation injection attack graph based recommendation systems recommendation system vulnerable injection attack malicious user due fundamental openness',
 'one vulnerability fake co visitation injection attack significantly impact recommendation system since modifies system according attacker s wish',
 'date detection co visitation injection attack challenging a one choice attribute representation node hard two practical evidence analyzing detecting anomaly real world data insufficient three challenging filter original injected co visitation data term node behavior',
 'paper investigates unified detection framework combine attribute network structure information synergistically detect outlier node based cur decomposition residual analysis',
 'first co visitation graph constructed using association rule attribute representation node developed',
 'then attribute network structure information blended order identify suspicious node',
 'extensive experiment synthetic re

Print out the similarity score and most similar sentences

In [113]:
#find and print the most similar sentences to 'input_sentence' using the Word2Vec model 'wv'.
input_sentence_index = 1
input_sentence = sentences1[input_sentence_index]
similar_sentences = wv.most_similar(input_sentence, topk=10)

# Print the most similar sentences
print(f"Most similar sentences to '{input_sentence}':")
for sentence, similarity in similar_sentences:
    print(f"Similarity: {similarity:.4f} - Sentence: '{sentence}'")

Most similar sentences to 'one vulnerability fake co visitation injection attack significantly impact recommendation system since modifies system according attacker s wish':
Similarity: 0.9664 - Sentence: 'key distinction prior work article considers sensor node aided backscatter communication allow communicate negligible energy cost using ambient radio frequency rf signal'
Similarity: 0.2360 - Sentence: 'fpga based updatable packet classification using tss combined bit selecting tree openflow switch deployed sdn enable wide spectrum non traditional application'
Similarity: 0.2205 - Sentence: 'structure configuration proposed mechanism explained'
Similarity: 0.2099 - Sentence: 'furthermore existing nmf model unable automatically determine rank latent matrix'
Similarity: 0.1956 - Sentence: 'nonorthogonal multiple access enabled two way relay system using signal alignment conventional two way relay nonorthogonal multiple access twr noma system diversity order equal zero even perfect succ

In [61]:
from gensim.models import Word2Vec, KeyedVectors, TfidfModel
from gensim.parsing.preprocessing import STOPWORDS
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cosine
import numpy as np

In [62]:
import gensim.downloader as api
word2vec_model = api.load("glove-wiki-gigaword-200")

[==================================================] 100.0% 252.1/252.1MB downloaded


In [114]:
combined_sentences = [' '.join(sentences) for sentences in sentences]
word_lists = [sentence.lower().split() for sentence in combined_sentences]


In [124]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np
def tf_idf_v2(sent):
    dct = Dictionary(sent)
    corpus = [dct.doc2bow(line) for line in sent]
    tf_idf_model = TfidfModel(corpus)
    vector = tf_idf_model[corpus]
    d = {dct.get(id): value for doc in vector for id, value in doc}
    sents_emd = []
    no_of_sent = sum(1 for i in sent)
    for i in range(no_of_sent):
        sent_emd = []
        for j in range(len(sent[i])):
            word = sent[i][j]
            if word in word2vec_model:
                emd = d[word]*word2vec_model[word]
                sent_emd.append(emd)
        sent_emd_np = np.array(sent_emd)
        sum_ = sent_emd_np.sum(axis=0)
        result = sum_/np.sqrt((sum_**2).sum())
        sents_emd.append(result)

    return sents_emd

In [73]:
sentences_emd2 = tf_idf_v2(word_lists)
print(len(sentences_emd2))
d2 = cosine(sentences_emd2[0],sentences_emd2[2])
print(d2)

30
0.16141235828399658


In [115]:
sent = test['sentences'].tolist()

In [116]:
combined_sent = [' '.join(sentences) for sentences in sent]
words = [sentence.lower().split() for sentence in combined_sent]

In [110]:
from sklearn.metrics.pairwise import cosine_similarity

In [128]:
test_sentence_embedding = tf_idf_v2(words)
#sentences_emd2_2d = [sentence_embedding for sentence_embedding in sentences_emd2]
cosine_similarities = cosine_similarity(test_sentence_embedding[1].reshape(1, -1), sentences_emd2)
most_similar_indices = cosine_similarities.argsort()[0][::-1]
most_similar_sentences = [word_lists[i] for i in most_similar_indices[:5]]

# Print the most similar sentences
for i, sentence in enumerate(most_similar_sentences):
    print(f"Similar Sentence {i + 1}: {sentence}")

Similar Sentence 1: ['many', 'objective', 'optimization', 'based', 'federal', 'deep', 'generation', 'model', 'enhancing', 'data', 'processing', 'capability', 'iot', 'rapid', 'progress', 'artificial', 'intelligence', 'expands', 'wide', 'applicability', 'internet', 'things', 'iot', 'meanwhile', 'data', 'insufficient', 'data', 'source', 'privacy', 'key', 'supply', 'chain', 'challenge', 'facing', 'iot', 'especially', 'healthcare', 'industry', 'address', 'problem', 'healthcare', 'iot', 'article', 'propose', 'skin', 'cancer', 'detection', 'model', 'based', 'federated', 'learning', 'integrated', 'deep', 'generation', 'model', 'first', 'employ', 'dual', 'generative', 'adversarial', 'network', 'address', 'problem', 'insufficient', 'data', 'addition', 'improve', 'quality', 'generated', 'image', 'synchronously', 'optimize', 'sharpness', 'image', 'frechet', 'inception', 'distance', 'image', 'diversity', 'loss', 'using', 'knee', 'point', 'driven', 'evolutionary', 'algorithm', 'knea', 'then', 'prote